### *Maciej Pawlikowski*

# Projekt: system rekomendujący

## Dane

Celem projektu było stworzenie systemu polecającego filmy. Wszystkie eksperymenty prowadziłem na danych MovieLens https://grouplens.org/datasets/movielens/:
- MovieLens Latest Small: http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
- MovieLens 1M Dataset: http://files.grouplens.org/datasets/movielens/ml-1m.zip
- MovieLens 10M Dataset: http://files.grouplens.org/datasets/movielens/ml-10m.zip

Dane zawierają oceny przyznane filmom przez użytkowników:
- Small: 9000 filmów, 700 użytkowników
- 1M: 6000 filmów, 4000 użytkowników
- 10M: 10000 filmów, 72000 użytkowników

## Podejście

Problem znalezienia filmu do polecenia użytkownikowi X można sprowadzić do problemu znalezienia oszacowań brakujących ocen tego użytkownika. Na podstawie ocen, które X wystawił, oraz ocen innych użytkowników znajdujemy przybliżone oceny filmów, których X nie ocenił.

Przetestowałem kilka metod szacujących oceny:
- Baseline Predictors
- User-User Collaborative Filtering
- Item-Item Collaborative Filtering
- Basic Slope One
- Weighted Slope One
- Bi-polar Slope One

Informacje o trzech pierwszych czerpałem z [1], o Slope One przeczytałem w [2].

[1] *Collaborative Filtering Recommender Systems*, Michael D. Ekstrand, John T. Riedl, Joseph A. Konstan http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf

[2] *Slope One Predictors for Online Rating-Based Collaborative Filtering*, Daniel Lemire, Anna Maclachlan http://lemire.me/fr/documents/publications/lemiremaclachlan_sdm05.pdf

## Opis metod

### Baseline Predictors

Dla każdego użytkownika $u$ i filmu $i$ jako przybliżenie oceny $i$ przez $u$ przyjmujemy

$$b_{u,i} = \mu + b_u + b_i,$$
gdzie
$$b_u = \frac{1}{|I_u|} \sum\limits_{i \in I_u} (r_{u,i} - \mu)$$
$$b_i = \frac{1}{|U_i|} \sum\limits_{u \in U_i} (r_{u,i} - b_u - \mu)$$
$\mu$ = średnia wszystkich ocen w zbiorze danych

$I_u$ = przedmioty ocenione przez $u$

$U_i$ = użytkownicy, którzy ocenili $i$

$r_{u,i}$ = ocena wystawiona $i$ przez $u$

$b_{u,i}$ to zatem średnia ocena znieształcona przez odchylenie od średniej właściwe ocenom wystawionym przez $u$ i ocenom wystawionym $i$.

### User-User Collaborative Filtering

Metoda szacuje ocenę na podstawie ocen wystawionych przez podobnych użytkowników. Najpierw obliczamy macierz podobieństw między użytkownikami. Przetestowałem dwie miary podobieństwa:
- miara kosinusowa: $$s(u,v) = \frac{\textbf{r}_u \cdot \textbf{r}_v}{||\textbf{r}_u||_2 ||\textbf{r}_v||_2}$$

  Można też wziąć pod uwagę fakt, że różni użytkownicy mają różne skale oceniania (np. ktoś w ogóle nie używa najwyższej oceny) odejmując od wszystkich ocen średnią danego użytkownika. Wtedy np. trójka wystawiona przez pesymistę może być blisko czwórki przeciętnego użytkownika.
  
  
- korelacja Pearsona: $$s(u,v) = \frac{\sum\limits_{i \in I_u \cap I_v} (r_{u,i}-\bar{r}_u)(r_{v,i}-\bar{r}_v)}{\sqrt{\sum\limits_{i \in I_u \cap I_v} (r_{u,i}-\bar{r}_u)^2} \sqrt{\sum\limits_{i \in I_u \cap I_v} (r_{v,i}-\bar{r}_v)^2}}$$

  Żeby uwiarygodnić nieco tę miarę wynik można dodatkowo uzależnić od liczby przedmiotów ocenionych przez obu użytkowników. Wtedy $s(u,v)$ mnożymy przez $\min(\frac{|I_u \cap I_v|}{threshold}, 1)$. W [1] proponują $threshold = 50$.
  
Po obliczeniu macierzy podobieństw wyznaczamy oceny w następujący sposób:

$$p_{u,i} = \bar{r}_u + \sigma_u \frac{\sum\limits_{u' \in N} s(u,u')(r_{u',i}-\bar{r}_{u'}) / \sigma_{u'}}{\sum\limits_{u' \in N} |s(u,u')|},$$
gdzie 

$\sigma_u$ = odchylenie standardowe ocen użytkownika $u$

$N$ = sąsiedztwo użytkownika $u$.

A zatem szacowana ocena opiera się na średniej ważonej ustandaryzowanych ocen bliskich najbliższych użytkowników. Stosując tę metodę za $N$ biorę pewne $k$ najbliższych $u$ użytkowników spośród takich, którzy ocenili przedmiot $i$.

### Item-Item Collaborating Filtering

Metoda szacuje ocenę wystawioną przez $u$ filmowi $i$ na podstawie ocen wystawionych przez $u$ filmom podobnym do $i$. Jak poprzednio, obliczamy macierz podobieństwa, tym razem dla filmów. Tutaj stosuję tylko miarę kosinusową (wg. [1] współczynnik Pearsona nie sprawdza się tutaj tak dobrze).

$$s(i,j) = \frac{\textbf{r}_i \cdot \textbf{r}_j}{||\textbf{r}_i||_2 ||\textbf{r}_j||_2}$$

Żeby lepiej porównywać różne skale oceniania różnych użytkowników, na czas obliczania podobieństw od wszystkich ocen możemy najpierw odjąć oceny bazowe $b_{u,i}$.

Szacujemy oceny tak:
$$p_{u,i} = \frac{\sum\limits_{j \in S} s(i,j)(r_{u,j}-b_{u,i})}{\sum\limits_{j \in S} |s(i,j)|} + b_{u,i},$$
gdzie

$S$ = pewne $k$ najbliższych filmowi $i$ filmów, spośród tych, które zostały ocenione przez $u$.

### Schematy Slope One

Schematy Slope One opierają się na "przesuwaniu" ocen. Próbując oszacować ocenę filmu $i$ dla użytkownika $u$ bierzemy pod uwagę średnie różnice między ocenami filmów ocenionych przez $u$ a ocenami filmu $i$.

Najpierw obliczamy macierz średnich różnic między ocenami filmów:

$$dev_{j,i} = \sum\limits_{u \in I_{j,i}} \frac{u_j - u_i}{|I_{j,i}|},$$
gdzie

$I_{j,i}$ = użytkownicy, którzy ocenili zarówno film $j$, jak i film $i$.


### Basic Slope One

Dla użytkownika $u$ ocenę filmu $j$ na podstawie oceny filmu $i$ można szacować jako $dev_{j,i} + u_i$. Podstawowy schemat Slope One to średnia z takich oszacowań:

$$p_{u,j} = \frac{1}{|R_j|} \sum\limits_{i \in R_j} (dev_{j,i} + u_i),$$
gdzie

$R_j = \lbrace i | i \in I_u, i \neq j, |I_{j,i}| > 0 \rbrace $.

W [2] jest zaproponowane przybliżenie tej metody, które znacznie zmniejsza koszt obliczania ocen. Dla odpowiednio gęstych danych $R_j$ prawie zawsze jest równe $I_u \setminus \lbrace j \rbrace$. W takim przypadku można formułę uprościć:

$$p_{u,j} = \bar{u} + \frac{1}{|R_j|} \sum\limits_{i \in R_j} dev_{j,i}.$$

### Weighted Slope One

Ważony Slope One stara się uwiarygodnić te szacowania, które powstały na podstawie ocen większej liczby użytkowników. Elementy macierzy $dev$ są więcej warte, gdy zostały obliczone z pięciuset ocen niż gdy z pięciu. Zamiast zwykłej średniej bierzemy średnią ważoną, z liczbami użytkowników jako wagami:

$$p_{u,j} = \frac{\sum\limits_{i \in S_u \setminus \lbrace j \rbrace} (dev_{j,i} + u_i) |I_{j,i}|}{\sum\limits_{i \in S_u \setminus \lbrace j \rbrace} |I_{j,i}|}$$

### Bi-polar Slope One

Ta metoda dzieli oceny każdego użytkownika na dwie grupy: oceny pozytywne i oceny negatywne. Za pozytywne są uznawane oceny wyższe niż średnia danego użytkownika. Nie wyciągamy żadnych wniosków z różnicy ocen filmu $i$ między użytkownikami $u$ i $v$ gdy $u$ lubi $i$, a $v$ nie. Budujemy dwie osobne macierze średnich różnic, jedną mierzącą różnice między ocenami pozytywnymi, drugą - między negatywnymi:

$$dev_{j,i}^{like} = \sum\limits_{u \in I_{j,i}^{like}} \frac{u_j - u_i}{|I_{j,i}^{like}|},$$
gdzie

$I_{j,i}^{like}$ = użytkownicy, którzy pozytywnie ocenili zarówno film $j$, jak i film $i$.

Analogicznie definiujemy $dev^{dislike}$.

Szacowanie dla użytkownika $u$ oceny filmu $j$ mając jego ocenę filmu $i$ zależy od tego, czy $u$ lubi $i$. Jeśli tak, to jest to $dev_{j,i}^{like} + u_i$, a jeśli nie, $dev_{j,i}^{dislike} + u_i$. Pełny wzór wygląda tak:

$$p_{u,j} = \frac{{\sum\limits_{i \in I_u^{like} \setminus \lbrace j \rbrace} (dev_{j,i}^{like} + u_i) |I_{j,i}^{like}|} + {\sum\limits_{i \in I_u^{dislike} \setminus \lbrace j \rbrace} (dev_{j,i}^{dislike} + u_i) |I_{j,i}^{dislike}|}}{{\sum\limits_{i \in I_u^{like} \setminus \lbrace j \rbrace} |I_{j,i}^{like}|} + {\sum\limits_{i \in I_u^{dislike} \setminus \lbrace j \rbrace} |I_{j,i}^{dislike}|}}.$$